In [11]:
# Load necessary libraries
library(tseries)
library(vars)
library(lmtest)
library(car)

# Load the data
data <- read.csv("./stationary_data.csv")

# Perform ADF tests on each variable
adf_var1 <- adf.test(data$var1, alternative = "stationary")
adf_var2 <- adf.test(data$var2, alternative = "stationary")
adf_var3 <- adf.test(data$var3, alternative = "stationary")

# Print p-values of ADF tests
cat("ADF Test P-Values:\n")
cat("var1:", adf_var1$p.value, "\n")
cat("var2:", adf_var2$p.value, "\n")
cat("var3:", adf_var3$p.value, "\n")

# Prepare data for VAR model
var_data <- data[, c("var1", "var2", "var3")]

# Select optimal lag
lag_selection <- VARselect(var_data, lag.max = 10)
print(lag_selection)
optimal_lag <- lag_selection$selection["AIC(n)"]

# Fit the VAR model
var_model <- VAR(var_data, p = optimal_lag)
summary(var_model)

# Extract residuals
residuals <- residuals(var_model)

# Apply tests to each equation in the VAR model
cat("\nP-Values of Residual Tests for Each Equation:\n")

# Loop through each equation
for (var in names(var_model$varresult)) {
  cat("\nTesting residuals for equation:", var, "\n")
  
  # Extract the sub-model (equation) as a linear model
  equation <- var_model$varresult[[var]]
  
  # 1. Raintest (linearity)
  raintest_result <- tryCatch(raintest(equation), error = function(e) NA)
  cat("Raintest p-value:", ifelse(is.na(raintest_result), "Error", raintest_result$p.value), "\n")
  
  # 2. Durbin-Watson Test (autocorrelation)
  dw_test <- dwtest(equation)
  cat("Durbin-Watson Test p-value:", dw_test$p.value, "\n")
  
  # 3. Breusch-Pagan Test (heteroscedasticity)
  bp_test <- bptest(equation)
  cat("Breusch-Pagan Test p-value:", bp_test$p.value, "\n")
  
  # 4. Shapiro-Wilk Test (normality of residuals)
  eq_residuals <- residuals[, var]
  shapiro_test <- shapiro.test(eq_residuals)
  cat("Shapiro-Wilk Test p-value:", shapiro_test$p.value, "\n")
}

# Explain var3 in terms of var1 and var2 using the VAR model
cat("\nShort-term VAR Model Explanation for var3:\n")
var3_equation <- var_model$varresult$var3
summary(var3_equation)

ADF Test P-Values:
var1: 0.04416789 
var2: 0.03063591 
var3: 0.02010764 
$selection
AIC(n)  HQ(n)  SC(n) FPE(n) 
     1      1      1      1 

$criteria
          1    2    3    4    5    6    7    8    9   10
AIC(n) -Inf -Inf -Inf -Inf -Inf -Inf -Inf -Inf -Inf -Inf
HQ(n)  -Inf -Inf -Inf -Inf -Inf -Inf -Inf -Inf -Inf -Inf
SC(n)  -Inf -Inf -Inf -Inf -Inf -Inf -Inf -Inf -Inf -Inf
FPE(n)    0    0    0    0    0    0    0    0    0    0




VAR Estimation Results:
Endogenous variables: var1, var2, var3 
Deterministic variables: const 
Sample size: 11 
Log Likelihood: -21.036 
Roots of the characteristic polynomial:
0.6107 0.6107 0.02007
Call:
VAR(y = var_data, p = optimal_lag)


Estimation results for equation var1: 
var1 = var1.l1 + var2.l1 + var3.l1 + const 

        Estimate Std. Error t value Pr(>|t|)
var1.l1   0.7242     0.5977   1.212    0.265
var2.l1  -0.4322     0.6445  -0.670    0.524
var3.l1  -1.6227     1.8684  -0.868    0.414
const     1.4450     1.5061   0.959    0.369


Residual standard error: 4.514 on 7 degrees of freedom
Multiple R-Squared: 0.2987,	Adjusted R-squared: -0.001786 
F-statistic: 0.9941 on 3 and 7 DF,  p-value: 0.4494 


Estimation results for equation var2: 
var2 = var1.l1 + var2.l1 + var3.l1 + const 

         Estimate Std. Error t value Pr(>|t|)    
var1.l1  0.611402   0.007093  86.198 7.45e-12 ***
var2.l1 -0.099487   0.007649 -13.006 3.70e-06 ***
var3.l1 -0.026990   0.022174  -1.217    0.


P-Values of Residual Tests for Each Equation:

Testing residuals for equation: var1 


Warning message in pf(rain, df[1], df[2], lower.tail = FALSE):
"NaNs produced"


Raintest p-value: NaN NaN NaN Error NaN 
Durbin-Watson Test p-value: 0.3301547 
Breusch-Pagan Test p-value: 0.4448296 
Shapiro-Wilk Test p-value: 0.2270046 

Testing residuals for equation: var2 


Warning message in pf(rain, df[1], df[2], lower.tail = FALSE):
"NaNs produced"


Raintest p-value: NaN NaN NaN Error NaN 
Durbin-Watson Test p-value: 0.04741422 
Breusch-Pagan Test p-value: 0.8315195 
Shapiro-Wilk Test p-value: 0.4877728 

Testing residuals for equation: var3 


Warning message in pf(rain, df[1], df[2], lower.tail = FALSE):
"NaNs produced"


Raintest p-value: NaN NaN NaN Error NaN 
Durbin-Watson Test p-value: 0.2249827 
Breusch-Pagan Test p-value: 0.6282716 
Shapiro-Wilk Test p-value: 0.02029986 

Short-term VAR Model Explanation for var3:



Call:
lm(formula = y ~ -1 + ., data = datamat)

Residuals:
    Min      1Q  Median      3Q     Max 
-1.3510 -0.9498 -0.4705  0.4075  2.7969 

Coefficients:
        Estimate Std. Error t value Pr(>|t|)
var1.l1  0.16370    0.21946   0.746    0.480
var2.l1 -0.05616    0.23667  -0.237    0.819
var3.l1 -0.11768    0.68607  -0.172    0.869
const    0.38004    0.55301   0.687    0.514

Residual standard error: 1.657 on 7 degrees of freedom
Multiple R-squared:  0.1835,	Adjusted R-squared:  -0.1664 
F-statistic: 0.5243 on 3 and 7 DF,  p-value: 0.6793


In [13]:
# Make a prediction using the VAR model
pred <- predict(var_model, n.ahead = 5)
cat("\nPrediction for the next period:\n")
print(pred)


Prediction for the next period:


$var1
          fcst     lower     upper        CI
[1,] 1.1803319 -7.666274 10.026938  8.846606
[2,] 0.3060844 -9.174920  9.787089  9.481005
[3,] 0.8409995 -9.165647 10.847646 10.006646
[4,] 1.4244552 -8.793691 11.642602 10.218146
[5,] 1.5090242 -8.735515 11.753564 10.244540

$var2
          fcst     lower    upper        CI
[1,] 2.0007860  1.895797 2.105775 0.1049888
[2,] 0.4873962 -4.845373 5.820165 5.3327691
[3,] 0.1119921 -5.521439 5.745424 5.6334315
[4,] 0.4769505 -5.474915 6.428816 5.9518651
[5,] 0.7943696 -5.257574 6.846313 6.0519437

$var3
          fcst     lower    upper       CI
[1,] 0.6958210 -2.552561 3.944203 3.248382
[2,] 0.3790132 -3.064102 3.822128 3.443115
[3,] 0.3581745 -3.113907 3.830256 3.472081
[4,] 0.4692755 -3.036481 3.975032 3.505756
[5,] 0.5312163 -2.980800 4.043233 3.512016

